In [1]:
import csv
import pandas as pd

In [63]:
base_raw = pd.read_csv('data/grand_list_top_10_2014_2017.csv', dtype={
    'FIPS': str,
})
base_raw['FIPS'] = base_raw['FIPS'].apply(lambda fips: '0' + fips)
newdata_raw = pd.read_csv('raw/grand-list-top-10-2019.csv')

## Read Grand list and create a crosswalk town to grand lists

In [64]:
MGL = pd.read_csv('raw/municipal_grand_list_data_2001-2017.csv')

town2netgrandlist = MGL[
    (MGL.Year == 'SFY 2016-2017') & (MGL['Variable'] == 'Net Grand List')
].filter(['Town', 'Value']).set_index('Town').to_dict()['Value']

town2totalgrandlist = MGL[
    (MGL.Year == 'SFY 2016-2017') & (MGL['Variable'] == 'Total Gross Grand List')
].filter(['Town', 'Value']).set_index('Town').to_dict()['Value']

## Clean up Top 10 - 2019 data

In [65]:
newdata = newdata_raw.fillna(-9999)

# Clean up town names
newdata.Town = newdata.Town.apply(str.title)

# Remove duplicate towns
newdata.drop_duplicates(['Town'], inplace=True)

## Copy 2018 into 2019 and update whenever possible

In [73]:
rows2019 = base_raw[base_raw['Town Profile Year'] == 2018].copy(deep=True)
rows2019['Town Profile Year'] = 2019

def update_row(r):
    town = r.Town
    
    if newdata[newdata.Town == r.Town].size > 0:
        entry = newdata[newdata.Town == r.Town]['Business Name #' + str(r.Rank)].iloc[0]
        if entry != -9999:
            r['Year Submitted'] = 2019
            r['Year'] = 2018
            r['Entry'] = entry
            
            if r.Variable == 'Grand List Value':
                r['Value'] = newdata[newdata.Town == r.Town]['Grand List Value #' + str(r.Rank)].iloc[0]
            elif r.Variable == 'Percent of Net Grand List':
                r['Value'] = round(newdata[newdata.Town == r.Town]['Grand List Value #' + str(r.Rank)].iloc[0] / town2netgrandlist[r.Town] * 100, 1)
            elif r.Variable == 'Percent of Total Grand List':
                r['Value'] = round(newdata[newdata.Town == r.Town]['Grand List Value #' + str(r.Rank)].iloc[0] / town2totalgrandlist[r.Town] * 100, 1)
            elif r.Variable == 'Percent of Top 10 Total Grand List':
                r['Value'] = '-6666'
            else:
                r['Value'] = '-6666'
    return r
    
rows2019 = rows2019.apply(update_row, axis=1)

## Concat dataframes, and done!

In [74]:
pd.concat([
    base_raw,
    rows2019
]).to_csv('data/grand-list-top-10-profiles2019.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

,Town,FIPS,Year,Year Submitted,Town Profile Year,Entry,Rank,Variable,Measure Type,Value
13520,Andover,0901301080,2018,2019,2019,Connecticut Light & Power,1,Grand List Value,Number,6.322860e+06
13521,Andover,0901301080,2018,2019,2019,Whispering Hills LLC,2,Grand List Value,Number,1.295300e+06
13522,Andover,0901301080,2018,2019,2019,Marita LLC,3,Grand List Value,Number,9.711000e+05
13523,Andover,0901301080,2018,2019,2019,Eastern CT Housing Organization,4,Grand List Value,Number,7.803000e+05
13524,Andover,0901301080,2018,2019,2019,84 Route six LLC,5,Grand List Value,Number,7.656000e+05
13525,Andover,0901301080,2018,2019,2019,Hillside Self Storage Center LLC,6,Grand List Value,Number,6.540000e+05
13526,Andover,0901301080,2018,2019,2019,Marcia Kenneth R & Katherine,7,Grand List Value,Number,5.826000e+05
13527,Andover,0901301080,2018,2019,2019,Beering Robert W,8,Grand List Value,Number,5.517000e+05
13528,Andover,0901301080,2018,2019,2019,Andover Plaza,9,Grand List Value,Number,5.261000e+05
13529,Andover,0901301080,2018,2019,2019,Hatem Pamela B,10,Grand List Value,Number,5.165000e+05


{'Andover': 265663230.0,
 'Ansonia': 897566947.0,
 'Ashford': 298676523.0,
 'Avon': 2592702830.0,
 'Barkhamsted': 347111840.0,
 'Beacon Falls': 489510636.0,
 'Berlin': 2194716070.0,
 'Bethany': 553020998.0,
 'Bethel': 1917648980.0,
 'Bethlehem': 369816439.0,
 'Bloomfield': 2038141920.0,
 'Bolton': 429848968.0,
 'Bozrah': 220302064.0,
 'Branford': 3505790076.0,
 'Bridgeport': 6065560261.0,
 'Bridgewater': 391337712.0,
 'Bristol': 3842668911.0,
 'Brookfield': 2269353833.0,
 'Brooklyn': 542832084.0,
 'Burlington': 907063811.0,
 'Canaan': 171596930.0,
 'Canterbury': 357741392.0,
 'Canton': 1107652979.0,
 'Chaplin': 161525200.0,
 'Cheshire': 2750332351.0,
 'Chester': 444233590.0,
 'Clinton': 1522190780.0,
 'Colchester': 1216010210.0,
 'Colebrook': 182148042.0,
 'Columbia': 476888490.0,
 'Cornwall': 404816100.0,
 'Coventry': 949436741.0,
 'Cromwell': 1322590255.0,
 'Danbury': 7026564235.0,
 'Darien': 8446673225.0,
 'Deep River': 490408404.0,
 'Derby': 718248343.0,
 'Durham': 694286939.0,
 'E